In [28]:
import time
import datetime

In [26]:
time.localtime()

time.struct_time(tm_year=2022, tm_mon=8, tm_mday=28, tm_hour=21, tm_min=55, tm_sec=16, tm_wday=6, tm_yday=240, tm_isdst=0)

In [29]:
datetime.datetime.now()

datetime.datetime(2022, 8, 28, 21, 55, 33, 619657)

In [12]:
# 오전 9 - 11
# 오후 13 - 18 
from time import localtime

#TODO 매 시간마다
#TODO 평일마다
if 9 <= localtime().tm_hour <= 11 or 13 <= localtime().tm_hour <= 18:
    print('자세 바로하세요') 

In [ ]:
# 년, 월, 일, 시간 출력
time.strftime("%Y년 %m월 %d일 %H:%M:%S")

In [ ]:
"""
시스템 정보
아래 스크립트 삭제시 JF deploy 실행이 안됩니다.
#JF_DEPLOYMENT_INPUT_DATA_INFO_START
{
    "deployment_input_data_form_list": [
        {
            "method": "POST",
            "location": "file",
            "api_key": "image",
            "value_type": "file",
            "category": "image",
            "category_description": ""
        }
    ]
}
#JF_DEPLOYMENT_INPUT_DATA_INFO_END
"""
import sys
sys.path.append('/addlib')
from deployment_api_deco import api_monitor
from flask import Flask, request, jsonify
from flask.views import MethodView
from flask_cors import CORS
import argparse
import requests
import base64
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from multiprocessing import Pool
import time


parser = argparse.ArgumentParser()
parser.add_argument('--prefix', type=str, default='/')
"""
배포 실행 명령어 관련 자동생성 영역
"""

params, _ = parser.parse_known_args()
params = vars(params)
app = Flask(__name__)
CORS(app, resources={r'/*': {"origins": '*'}})


model = load_model(os.path.abspath("./model.h5"), compile=False)

# def ran_int(a):
# #     # 이미지 저장
# #     file = request.files['image']
#     file_name = 'input.jpg'
# #     file.save(file_name)

#     # 이미지 읽어오기
#     img_grayscale = cv2.imread(file_name, 0)

#     # 이미지 전처리
#     img_resized = cv2.resize(img_grayscale, dsize=(28, 28), interpolation=cv2.INTER_CUBIC)
#     img = img_resized.reshape(-1, 28 * 28) / 255.0

#     # 이미지 판단
#     prediction = model.predict(img)[0]

#     # 결과
#     result = {}
#     for idx, val in enumerate(prediction):
#         result[idx] = str(val * 100)[:4] + '%'

#     # 최종 결과
#     final_result = list(prediction).index(max(prediction))

#     output = {
#         "text": [
#             {
#                 "final_result": final_result
#             }
#         ],
#     }

#     return jsonify(output)





class run_api(MethodView):
    def __init__(self):
        pass

    @api_monitor()
    def get(self):
        return "get method success"

    @api_monitor()
    def post(self):
#         # 병렬화
#         p = Pool(processes=2)
#         rls = p.map(ran_int, [1,2,3])
#         return(rls)
        start = time.time()

        # for 반복
        sum = 0
        for i in range(10000000):
            sum += 1


        # 이미지 저장
#         file = request.files['image']
        file_name = 'input.jpg'
        #file.save(file_name)

        # 이미지 읽어오기
        img_grayscale = cv2.imread(file_name, 0)

        # 이미지 전처리
        img_resized = cv2.resize(img_grayscale, dsize=(28, 28), interpolation=cv2.INTER_CUBIC)
        img = img_resized.reshape(-1, 28 * 28) / 255.0

        # 이미지 판단
        prediction = model.predict(img)[0]

        # 결과
        result = {}
        for idx, val in enumerate(prediction):
            result[idx] = str(val * 100)[:4] + '%'

        # 최종 결과
        final_result = list(prediction).index(max(prediction))

        output = {
            "text": [
                {
                    "final_result": final_result
                }
            ],
        }

        end = time.time()
        os.system(f"echo {end - start} >> log_requests_5000_1000.txt")

#         print(f"time is {end - start}")

        return jsonify(output)



app.add_url_rule(params["prefix"], view_func=run_api.as_view("run_api"))
if __name__ == "__main__":
    """
    모델 로드를 권장하는 위치
    사용자 영역
    """
    app.run('0.0.0.0',8555,threaded=True, debug=True)